In [71]:
import torch
from torch import nn
import torch.optim as optim
from torch.utils.data import DataLoader
import pandas as pd

In [72]:
# Read Data
spectrum_train = pd.read_excel('spectrum_train.xlsx')
spectrum_test = pd.read_excel('spectrum_valid.xlsx')
temp_train = pd.read_excel('temp_train.xlsx')
temp_test = pd.read_excel('temp_valid.xlsx')

In [74]:
# determine the supported device
def get_device():
    if torch.cuda.is_available():
        device = torch.device('cuda:0')
    else:
        device = torch.device('cpu') # don't have GPU 
    return device

# convert a df to tensor to be used in pytorch
def df_to_tensor(df):
    device = get_device()
    return torch.from_numpy(df.values).float().to(device)

In [75]:
input_data = df_to_tensor(spectrum_train)
output_data = df_to_tensor(temp_train)

In [76]:
class Net(nn.Module):
    def __init__(self, input_size, output_size):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(input_size, 128)
        self.fc2 = nn.Linear(128, output_size)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [77]:
input_size = 66
output_size = 11
model = Net(input_size=input_size, output_size=output_size)

In [78]:
criterion = nn.MSELoss()  # Mean Squared Error loss for regression
optimizer = optim.Adam(model.parameters(), lr=0.001)  # Adam optimizer

In [79]:
num_epochs = 100
batch_size = 32
num_batches = len(input_data) // batch_size

for epoch in range(num_epochs):
    for batch in range(num_batches):
        start = batch * batch_size
        end = start + batch_size

        inputs = input_data[start:end]
        targets = output_data[start:end]

        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)

        # Compute loss
        loss = criterion(outputs, targets)

        # Backpropagation and optimization
        loss.backward()
        optimizer.step()

    # Print the loss for this epoch
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {loss.item():.6f}")

Epoch 1/100, Loss: 286210.187500
Epoch 2/100, Loss: 285488.781250
Epoch 3/100, Loss: 284196.781250
Epoch 4/100, Loss: 282300.406250
Epoch 5/100, Loss: 279803.875000
Epoch 6/100, Loss: 276724.625000
Epoch 7/100, Loss: 273086.562500
Epoch 8/100, Loss: 268917.531250
Epoch 9/100, Loss: 264248.468750
Epoch 10/100, Loss: 259112.265625
Epoch 11/100, Loss: 253543.593750
Epoch 12/100, Loss: 247578.343750
Epoch 13/100, Loss: 241253.609375
Epoch 14/100, Loss: 234606.953125
Epoch 15/100, Loss: 227676.562500
Epoch 16/100, Loss: 220500.656250
Epoch 17/100, Loss: 213117.500000
Epoch 18/100, Loss: 205565.000000
Epoch 19/100, Loss: 197880.687500
Epoch 20/100, Loss: 190101.265625
Epoch 21/100, Loss: 182262.625000
Epoch 22/100, Loss: 174399.562500
Epoch 23/100, Loss: 166545.609375
Epoch 24/100, Loss: 158732.890625
Epoch 25/100, Loss: 150991.984375
Epoch 26/100, Loss: 143351.687500
Epoch 27/100, Loss: 135839.046875
Epoch 28/100, Loss: 128479.156250
Epoch 29/100, Loss: 121295.023438
Epoch 30/100, Loss: 114

In [80]:
predictions = model(df_to_tensor(spectrum_test))

In [81]:
predictions

tensor([[431.1968, 431.7384, 432.9923,  ..., 431.8170, 431.0523, 430.2775],
        [431.1968, 431.7384, 432.9923,  ..., 431.8170, 431.0523, 430.2775],
        [431.1968, 431.7384, 432.9923,  ..., 431.8170, 431.0523, 430.2775],
        ...,
        [431.1968, 431.7384, 432.9923,  ..., 431.8170, 431.0523, 430.2775],
        [431.1968, 431.7384, 432.9923,  ..., 431.8170, 431.0523, 430.2775],
        [431.1968, 431.7384, 432.9923,  ..., 431.8170, 431.0523, 430.2775]],
       grad_fn=<AddmmBackward0>)